In [1]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import argparse
import pandas as pd
from eofs.xarray import Eof
from sklearn import preprocessing
from sklearn.decomposition import PCA

from lwp_nd import lwp_nd_input_ICON
from PCA_radiances import PCA_calculation, variance_sklearn_plot, dataframe_csv, convert_3D

from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, check_scoring, mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics


In [3]:
path_ICON = "/home/jvillarreal/Documents/phd/dataset/data_rttov_T12.nc"
path_output = "/home/jvillarreal/Documents/phd/output/ML_output"
path_OUTPUT_RTTOV = "/home/jvillarreal/Documents/phd/github/output-rttov/output-test-2-modis.nc"
rttov_path_refl_emmis = "/home/jvillarreal/Documents/phd/github/output-rttov/rttov-131-data-icon-1to19-26-T12.nc"
rttov_path_rad = "/home/jvillarreal/Documents/phd/github/output-rttov/rttov-13-data-icon-1-to-36-not-flip.nc"
path_rttov_test = "/home/jvillarreal/Documents/phd/github/output-rttov/rttov-131-data-icon-1to36-T09.nc"
path_ICON_test = "/home/jvillarreal/Documents/phd/dataset/data_rttov_T09.nc"
path_ICON_test2 = "/home/jvillarreal/Documents/phd/dataset/data_rttov_T09.nc"



In [14]:
ds_T09, p_2013, T_2013, q_2013, max_cdnc_2013_cm, lwp_2013, lat, lon, height= lwp_nd_input_ICON(path_output = path_output, path_ICON = path_ICON_test) #obtain more outputs of it


<xarray.Dataset>
Dimensions:       (lat: 637, lon: 589, height: 150)
Coordinates:
  * height        (height) float64 1.0 2.0 3.0 4.0 ... 147.0 148.0 149.0 150.0
  * lat           (lat) float64 47.5 47.51 47.52 47.53 ... 54.47 54.48 54.5
  * lon           (lon) float64 4.5 4.517 4.534 4.551 ... 14.45 14.46 14.48 14.5
Data variables: (12/21)
    FR_LAND       (lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0
    cct           (lat, lon) float32 0.0 0.0 0.0 0.0 ... 7.503e+04 7.502e+04 0.0
    clc           (height, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cli           (height, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    clw           (height, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    clwvi         (lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.2108 0.1401 0.03157
    ...            ...
    tas           (lat, lon) float32 287.9 287.9 287.9 ... 279.5 279.4 279.4
    topography_c  (lat, lon) float32 411.2 411.7 411.7 393.6 ... 0.0 0.0 0

/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:75: RuntimeWarning: overflow encountered in exp
  esat_2013 = (0.611* np.exp(17.3*T_c))/(T_c+237.3)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:77: RuntimeWarning: overflow encountered in multiply
  esat_2013 = esat_2013*1000.0
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:80: RuntimeWarning: divide by zero encountered in true_divide
  RH_2013 = 100*(r_2013/qs_2013)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:80: RuntimeWarning: overflow encountered in true_divide
  RH_2013 = 100*(r_2013/qs_2013)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:80: RuntimeWarning: overflow encountered in multiply
  RH_2013 = 100*(r_2013/qs_2013)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:81: RuntimeWarning: invalid value encountered in multiply
  pv_2013 = (esat_2013*RH_2013)/100.0
/home/jvillarreal/Documents/ph

max and min cdnc nan nan
1 (150, 628, 589) (628, 589) (628, 589)
2 (55483800,) (369892,) (369892,)
nan debe ser igual to --
------------
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]


/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:39: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im = ax.pcolormesh(x, y, z, cmap ='jet', vmin=vmin, vmax = vmax ) #cmap=cmap.N, norm=norm)


===============T_2013 (height 120, lat 57, lon 227) cm: 276.151153564453 ==  276.37033
===============T_c (height 120, lat 57, lon 227) cm: 3.00115356445315==  3.220337
===============esat_2013 (height 120, lat 57, lon 227) 758.360598313415 ==  3.983164e+24
===============p_2013 (height 120, lat 57, lon 227)75935.328125 ==  76395.13
===============qs_2013 (height 120, lat 57, lon 227) 0.00621186875461262  ==  3.243044e+19
===============r_2013 (height 120, lat 57, lon 227) 0.0062973626597643 ==  0.0049734227
===============RH_2013 (height 120, lat 57, lon 227) 101.376299283339 ==  1.5335662e-20
===============pv_2013 (height 120, lat 57, lon 227) 768.797909793127 ==  610.84454
===============pd_2013 (height 120, lat 57, lon 227) 75166.5302152069 ==  75784.29
===============rho_2013 (height 120, lat 57, lon 227) 0.954250058491486 ==  0.9600416
===============cdnc_2013_cm (height 120, lat 57, lon 227) 15.5508091629487 ==  0.0


In [4]:
ds_T12, p_2013, T_2013, q_2013, max_cdnc_2013_cm, lwp_2013, lat, lon, height= lwp_nd_input_ICON(path_output = path_output, path_ICON = path_ICON) #obtain more outputs of it


<xarray.Dataset>
Dimensions:       (lat: 637, lon: 589, height: 150)
Coordinates:
  * height        (height) float64 1.0 2.0 3.0 4.0 ... 147.0 148.0 149.0 150.0
  * lat           (lat) float64 47.5 47.51 47.52 47.53 ... 54.47 54.48 54.5
  * lon           (lon) float64 4.5 4.517 4.534 4.551 ... 14.45 14.46 14.48 14.5
Data variables: (12/21)
    FR_LAND       (lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0
    cct           (lat, lon) float32 0.0 0.0 0.0 0.0 ... -999.0 7.681e+04 0.0
    clc           (height, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cli           (height, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    clw           (height, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    clwvi         (lat, lon) float32 0.0 8.124e-06 8.124e-06 ... 0.01018 0.04625
    ...            ...
    tas           (lat, lon) float32 289.8 289.9 289.9 ... 279.3 279.4 279.3
    topography_c  (lat, lon) float32 411.2 411.7 411.7 393.6 ... 0.0 0.0 0.

/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:75: RuntimeWarning: overflow encountered in exp
  esat_2013 = (0.611* np.exp(17.3*T_c))/(T_c+237.3)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:77: RuntimeWarning: overflow encountered in multiply
  esat_2013 = esat_2013*1000.0
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:80: RuntimeWarning: divide by zero encountered in true_divide
  RH_2013 = 100*(r_2013/qs_2013)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:80: RuntimeWarning: overflow encountered in true_divide
  RH_2013 = 100*(r_2013/qs_2013)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:80: RuntimeWarning: overflow encountered in multiply
  RH_2013 = 100*(r_2013/qs_2013)
/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:81: RuntimeWarning: invalid value encountered in multiply
  pv_2013 = (esat_2013*RH_2013)/100.0
/home/jvillarreal/Documents/ph

max and min cdnc nan nan
1 (150, 628, 589) (628, 589) (628, 589)
2 (55483800,) (369892,) (369892,)
nan debe ser igual to 0.6855309375
------------
 [           nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            na

/home/jvillarreal/Documents/phd/github/ICON_RTTOV/code_test/lwp_nd.py:39: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im = ax.pcolormesh(x, y, z, cmap ='jet', vmin=vmin, vmax = vmax ) #cmap=cmap.N, norm=norm)


===============T_2013 (height 120, lat 57, lon 227) cm: 276.151153564453 ==  276.40933
===============T_c (height 120, lat 57, lon 227) cm: 3.00115356445315==  3.2593384
===============esat_2013 (height 120, lat 57, lon 227) 758.360598313415 ==  7.819643e+24
===============p_2013 (height 120, lat 57, lon 227)75935.328125 ==  76403.13
===============qs_2013 (height 120, lat 57, lon 227) 0.00621186875461262  ==  6.3659924e+19
===============r_2013 (height 120, lat 57, lon 227) 0.0062973626597643 ==  0.004533605
===============RH_2013 (height 120, lat 57, lon 227) 101.376299283339 ==  7.1216e-21
===============pv_2013 (height 120, lat 57, lon 227) 768.797909793127 ==  556.8837
===============pd_2013 (height 120, lat 57, lon 227) 75166.5302152069 ==  75846.25
===============rho_2013 (height 120, lat 57, lon 227) 0.954250058491486 ==  0.960264
===============cdnc_2013_cm (height 120, lat 57, lon 227) 15.5508091629487 ==  0.0


In [18]:
ds_T12["ta"].values.min(),ds_T12["ta"].values.max()

(0.0, 295.99216)

In [19]:
ds_T09["ta"].values.min(),ds_T09["ta"].values.max()

(0.0, 292.963)

In [20]:
ds_T12["hus"].values.min(),ds_T12["hus"].values.max()

(0.0, 0.011469095)

In [21]:
ds_T09["hus"].values.min(),ds_T09["hus"].values.max()

(0.0, 0.010540523)

In [22]:
ds_T12["pres"].values.min(),ds_T12["pres"].values.max()

(0.0, 103661.13)

In [23]:
ds_T09["pres"].values.min(),ds_T09["pres"].values.max()

(0.0, 103740.53)

In [24]:
ds_T12["clwvi"].values.min(),ds_T12["clwvi"].values.max()

(0.0, 6.158819)

In [25]:
ds_T09["clwvi"].values.min(),ds_T09["clwvi"].values.max()

(0.0, 1.4530537)